## Import Required Libraries

In [15]:
import torch
from IPython.display import Image
import matplotlib.pyplot as plt
import shutil 
import os
import warnings
import cv2
import stat

# Suppress the specific FutureWarning
warnings.filterwarnings("ignore", message = ".*torch.cuda.amp.autocast.*")

## Convert Videos to Frames

In [ ]:
# Extract frames from a video at a specified interval and save them in a specified output_folder
def convert_video(video_path, output_folder, interval):

    # Open the video file
    video_file = cv2.VideoCapture(video_path)

    # Get video frame rate
    fps = int(video_file.get(cv2.CAP_PROP_FPS))

    # Compute the frame interval (in frames, not seconds)
    frame_interval = fps * interval

    # Initialize frame count counter
    frame_count = 0

    while video_file.isOpened():

        # Read the next frame
        ret, frame = video_file.read()

        # If the frame is not read correctly, break the loop
        if not ret:
            break

        # Save frames at the specified interval
        if frame_count % frame_interval == 0:
            # Convert frame count to seconds
            timestamp = int(frame_count / fps)

            # Establihs a filename as the current timestamp 
            filename = os.path.join(output_folder, f"{timestamp:05d}.jpg")

            # Write the frame to the output folder
            cv2.imwrite(filename, frame)

        # Increase the frame count
        frame_count += 1

    video_file.release()
    


# Clears the read-only attribute before deleting a file
def remove_readonly(func, path, _):
    os.chmod(path, stat.S_IWRITE)
    func(path)



# Process all videos in input_dir and save frames in output_dir
def process_videos(input_dir, output_dir, interval_seconds, data_type, max_videos = None):

    # If its exists, remove the output directory to ensure a fresh start
    if os.path.exists(output_dir):
        # Delete the directory and all contents
        shutil.rmtree(output_dir, onerror = remove_readonly)  

    # Create a new output directory
    os.makedirs(output_dir, exist_ok = True)

    # Initialize saved video count for a summary at the end
    saved_videos = 0

    # Loop through all video files in input directory
    for video_name in os.listdir(input_dir):

        # Stop after processing the specified number of videos
        if max_videos != None and saved_videos >= max_videos:
            break

        # Establish a video path with the full filepath
        video_path = os.path.join(input_dir, video_name)

        # Check if the video file id valid
        if os.path.isfile(video_path) and video_name.lower().endswith((".mp4")):

            # Extract video name without .mp4 extension
            video_id = os.path.splitext(video_name)[0]  

            # Establish a separate folder for each video
            output_folder = os.path.join(output_dir, video_id)  

            # Check if the output folder exists, if not create it
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            # Convert video into frames with designated interval
            convert_video(video_path, output_folder, interval_seconds)

            # Incrase saved videos count
            saved_videos += 1

    # Print summary counts of saved frames
    print(f"Processed {saved_videos} {data_type} videos to {output_dir}\n")

In [ ]:
# Define input directories
train_input_dir = "../data/raw_data/train/"
test_input_dir = "../data/raw_data/test/"

# Define output directories
train_output_dir = "../data/processed_data/train/"
test_output_dir = "../data/processed_data/test/"

# Set frame extraction interval
interval_seconds = 1

# Create data_types
training_data_type = "Training"
testing_data_type = "Testing"

# Process train and test videos
process_videos(train_input_dir, train_output_dir, interval_seconds, training_data_type)
#process_videos(test_input_dir, test_output_dir, interval_seconds, testing_data_type)

Processed 1500 Training videos to ../data/processed_data/train/

Processed 1344 Testing videos to ../data/processed_data/test/

